最初版本

In [55]:
import xlwings as xw,os

# 清理财务数据函数，去除空格，将null的值替换为0
def DataClean(data):
    for i in data:
        if i[0] is None:continue
        i[0] = i[0].replace(" ","")
        if i[1] is None:i[1]= 0
    return data

def ReadFinanceReport(path):
    Book = xw.Book(fakeFilePath)
    sht = Book.sheets[0]

    # 读表头信息
    financeReportInf = sht.range('A3:B13').value + sht.range('C3:D12').value
    # 读资产负债表信息
    BalanceSheet = Book.sheets[1]
    BalanceData = BalanceSheet.range('B5:C54').value + BalanceSheet.range('E5:F51').value
    # 读损益表信息
    IncomeSheet = Book.sheets[2]
    IncomeData = IncomeSheet.range('B4:C51').value
    # 读现金流量表信息
    CashflowSheet = Book.sheets[3]
    CashflowData = CashflowSheet.range('B4:C38').value
    # 清洗数据，数据词典化
    BalanceData,IncomeData,CashflowData = DataClean(BalanceData),DataClean(IncomeData),DataClean(CashflowData)
    BalanceData,IncomeData,CashflowData,financeReportInf = dict(BalanceData),dict(IncomeData),dict(CashflowData),dict(financeReportInf)
    FinalFinanceReport = [financeReportInf,BalanceData, IncomeData,CashflowData]
    return FinalFinanceReport

fakeFilePath = r'/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/广东周氏神龙电线制造有限公司非合并报表2021年报表.xls'
h1 = ReadFinanceReport(fakeFilePath)
h2 = {'h2': h1}
# ys = BalanceData.get('应收账款')


os路径检索模块

In [10]:
import os,re
projectPath = r'/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/'
dir_list = os.listdir(projectPath)
dir_pattern = r".*有限公司.*合并报表\d{4}年.*"
matches = []
for e in dir_list:
    match = re.findall(dir_pattern, e)
    if match:
        matches.append(projectPath + match[0])
print(matches)

['/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/广东周氏神龙电线制造有限公司非合并报表2022年报表.xls', '/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/广东周氏神龙电线制造有限公司非合并报表2020年报表.xls', '/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/广东周氏神龙电线制造有限公司非合并报表2023年5月报表.xls', '/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/广东周氏神龙电线制造有限公司非合并报表2021年报表.xls']


核心进程（待整合fi class）

In [57]:
import xlwings as xw,os,re,matplotlib

dates = {'第1季度':3,'上半年':6,'第3季度':9,'4月':4,'5月':5,'7月':7,'8月':8,'11月':11,'10月':10,'2月':2,'12月':12,'1月':1}
assetSubjects = ['货币资金','交易性金融资产','以公允价值计量且其变动计入当期损益的金融资产','衍生金融资产','其中：应收票据','应收账款','应收款项融资','预付款项','其他应收款','存货','合同资产','持有待售资产','一年内到期的非流动资产','其他流动资产','债权投资','其他债权投资','可供出售金融资产','持有至到期投资','长期应收款','长期股权投资','其他权益工具投资','其他非流动金融资产','投资性房地产','固定资产','在建工程','生产性生物资产','油气资产','使用权资产','无形资产','开发支出','商誉','长期待摊费用','递延所得税资产','其他非流动资产']
# 清理财务数据函数，去除空格，将null的值替换为0
def DataClean(data):
    for i in data:
        if i[0] is None:continue
        i[0] = i[0].replace(" ","").replace("*","").replace("'","").replace(",","").replace("，","").replace("其中：应收票据","应收票据")
        if i[1] is None:i[1] = 0
        if isinstance(i[1], (int, float)):i[1] = round(float(i[1]) ,2)
        if isinstance(i[1], (str)):i[1] = i[1].replace(" ","").replace("*","")
    return data

# 封包读取单个财报流程，输入path，输出dict格式
def ReadFinanceReport(path):
    Book = xw.Book(path)
    sht = Book.sheets[0]

    # 读表头信息
    financeReport = sht.range('A3:B13').value + sht.range('C3:D12').value
    # 读资产负债表信息
    BalanceSheet = Book.sheets[1]
    financeReport += BalanceSheet.range('B5:C64').value + BalanceSheet.range('E5:F64').value
    # 读损益表信息
    IncomeSheet = Book.sheets[2]
    financeReport += IncomeSheet.range('B4:C54').value
    # 读现金流量表信息
    CashflowSheet = Book.sheets[3]
    financeReport += CashflowSheet.range('B4:C71').value
    # 清洗数据，数据词典化
    financeReport = DataClean(financeReport)
    financeReport = dict(financeReport)

    Book.close()
    return financeReport

# 获取指定目录下财报excel详细路径，输入目录路径，输出财报路径列表
def GetFinanceReportsPath(projectPath):
    dir_list = os.listdir(projectPath)
    dir_pattern = r".*有限公司.*合并报表\d{4}年.*"
    matches = []
    for e in dir_list:
        match = re.findall(dir_pattern, e)
        if match:
            matches.append(projectPath + match[0])
    return matches

# 循环读取获得的excel路径，输入路径，输出一个包含字典的列表
def ReadAllFinanceReports(projectPath):
    pathes = GetFinanceReportsPath(projectPath)
    AllFinanceReportsList = []
    for path in pathes:
        singleFinanceReport = ReadFinanceReport(path)
        # 在字典中加了一个Head的键，值为一个列表，包括【客户名、年份及期次【年、月（年报月为null）】、报表期次类型】
        single_dates = singleFinanceReport.get('报表期次')
        single_type = singleFinanceReport.get('报表周期')
        int_year = int(single_dates[0:4])
        int_date = dates.get(single_dates[5:-1])
        int_dates = [int_year, int_date]
        single_name = singleFinanceReport.get('客户名称')
        singleFinanceReport['Head']=[single_name, int_dates, single_type]
        AllFinanceReportsList.append(singleFinanceReport)
    return AllFinanceReportsList

# 从小到大对列表进行排序，排序依据为字典中Head对应的年份（Head-dates-year）
def SortAll(all):
    sortedList = sorted(all, key = lambda x:x['Head'][1][0])
    return sortedList

    

# 主任务——输入
projectPath = r'/Users/kataru/Documents/办公文稿/230701 周氏神龙1000/'
allFinanceReports = ReadAllFinanceReports(projectPath)
allFinanceReports = SortAll(allFinanceReports)
print(allFinanceReports)



[{'客户名称': '广东周氏神龙电线制造有限公司', '报表周期': '年报', '报表期次': '2020年', '是否审计': '否', '审计意见': 0.0, '审计意见说明': 0.0, '是否控制名单': '否', '币种': '人民币', '版本': 'V[1]', '报表缺失原因': 0.0, '备注': 0.0, '客户编号': '700008131614', '报表性质': '非合并报表', '录入人': '刘嘉华', '审计日期': 0.0, '审计报告编号': 0.0, None: None, '会计事务所名称': 0.0, '汇率': 0.15, '报表状态': '已生效', '报表模板': '一般企业财务报表(2018版会计准则)', '货币资金': 3770116.25, '交易性金融资产': 570000.0, '以公允价值计量且其变动计入当期损益的金融资产': 0.0, '衍生金融资产': 0.0, '应收票据及应收账款': 31063142.65, '应收票据': 2980257.35, '应收账款': 28082885.3, '应收款项融资': 0.0, '预付款项': 0.0, '其他应收款': 9400.96, '其中：应收利息': 0.0, '应收股利': 0.0, '存货': 550625.78, '合同资产': 0.0, '持有待售资产': 0.0, '一年内到期的非流动资产': 0.0, '其他流动资产': 3234922.14, '流动资产合计': 39198207.78, '非流动资产：': 0.0, '债权投资': 0.0, '其他债权投资': 0.0, '可供出售金融资产': 0.0, '持有至到期投资': 0.0, '长期应收款': 0.0, '长期股权投资': 0.0, '其他权益工具投资': 0.0, '其他非流动金融资产': 0.0, '投资性房地产': 0.0, '其中：投资性房地产原值': 0.0, '投资性房地产累计折旧': 0.0, '投资性房地产减值准备': 0.0, '固定资产': 1526414.67, '其中：固定资产原值': 7091588.9, '固定资产累计折旧': 5565174.23, '固定资产减值准备': 0.0, '固定资产清理': 0.0, '在建工程': 0.0,

fi类及其方法构建的模块

In [54]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

filist = [{'客户名称': '广东周氏神龙电线制造有限公司', '报表周期': '年报', '报表期次': '2020年', '是否审计': '否', '审计意见': 0.0, '审计意见说明': 0.0, '是否控制名单': '否', '币种': '人民币', '版本': 'V[1]', '报表缺失原因': 0.0, '备注': 0.0, '客户编号': '700008131614', '报表性质': '非合并报表', '录入人': '刘嘉华', '审计日期': 0.0, '审计报告编号': 0.0, None: None, '会计事务所名称': 0.0, '汇率': 0.15, '报表状态': '已生效', '报表模板': '一般企业财务报表(2018版会计准则)', '货币资金': 3770116.25, '交易性金融资产': 570000.0, '以公允价值计量且其变动计入当期损益的金融资产': 0.0, '衍生金融资产': 0.0, '应收票据及应收账款': 31063142.65, '应收票据': 2980257.35, '应收账款': 28082885.3, '应收款项融资': 0.0, '预付款项': 0.0, '其他应收款': 9400.96, '其中：应收利息': 0.0, '应收股利': 0.0, '存货': 550625.78, '合同资产': 0.0, '持有待售资产': 0.0, '一年内到期的非流动资产': 0.0, '其他流动资产': 3234922.14, '流动资产合计': 39198207.78, '非流动资产：': 0.0, '债权投资': 0.0, '其他债权投资': 0.0, '可供出售金融资产': 0.0, '持有至到期投资': 0.0, '长期应收款': 0.0, '长期股权投资': 0.0, '其他权益工具投资': 0.0, '其他非流动金融资产': 0.0, '投资性房地产': 0.0, '其中：投资性房地产原值': 0.0, '投资性房地产累计折旧': 0.0, '投资性房地产减值准备': 0.0, '固定资产': 1526414.67, '其中：固定资产原值': 7091588.9, '固定资产累计折旧': 5565174.23, '固定资产减值准备': 0.0, '固定资产清理': 0.0, '在建工程': 0.0, '生产性生物资产': 0.0, '其中：生产性生物资产原值': 0.0, '生产性生物资产累计折旧': 0.0, '生产性生物资产减值准备': 0.0, '油气资产': 0.0, '其中：油气资产原值': 0.0, '油气资产累计折耗': 0.0, '油气资产减值准备': 0.0, '使用权资产': 0.0, '其中：使用权资产原值': 0.0, '使用权资产累计折旧': 0.0, '使用权资产减值准备': 0.0, '无形资产': 0.0, '其中：无形资产原值': 0.0, '无形资产累计摊销': 0.0, '无形资产减值准备': 0.0, '开发支出': 0.0, '商誉': 0.0, '长期待摊费用': 0.0, '递延所得税资产': 0.0, '其他非流动资产': 0.0, '非流动资产合计': 1526414.67, '资产总计': 40724622.45, '短期借款': 6000000.0, '交易性金融负债': 0.0, '以公允价值计量且其变动计入当期损益的金融负债': 0.0, '衍生金融负债': 0.0, '应付票据及应付账款': 17732605.75, '其中：应付票据': 3540019.43, '应付账款': 14192586.32, '预收款项': 0.0, '合同负债': 0.0, '应付职工薪酬': 342188.38, '应交税费': 278685.05, '其他应付款': 600808.67, '其中：应付利息': 0.0, '应付股利': 0.0, '持有待售负债': 0.0, '一年内到期的非流动负债': 0.0, '其他流动负债': 0.0, '流动负债合计': 24954287.85, '非流动负债：': 0.0, '长期借款': 0.0, '应付债券': 0.0, '其中：优先股': 0.0, '永续债': 0.0, '租赁负债': 0.0, '长期应付款': 0.0, '预计负债': 0.0, '递延收益': 0.0, '递延所得税负债': 0.0, '其他非流动负债': 0.0, '非流动负债合计': 0.0, '负债合计': 24954287.85, '所有者权益（或股东权益）：': 0.0, '实收资本（或股本）': 3000000.0, '其他权益工具': 0.0, '资本公积': 0.0, '减：库存股': 0.0, '其他综合收益': 0.0, '专项储备': 0.0, '盈余公积': 1871491.22, '未分配利润': 10898843.38, '少数股东权益': 0.0, '所有者权益（或股东权益）合计': 15770334.6, '负债和所有者权益（或股东权益）总计': 40724622.45, '一、营业收入': 68181835.95, '减：营业成本': 61609802.0, '税金及附加': 219171.47, '销售费用': 1131175.91, '管理费用': 3492356.69, '研发费用': 0.0, '财务费用': 514038.24, '其中:利息费用（本子科目指的是剔除利息资本化部分的；计入损益表的利息费用可通过“财务费用”填列）': 0.0, '利息收入': 0.0, '资产减值损失': 0.0, '信用减值损失': 0.0, '加：其他收益': 0.0, '投资收益（损失以“-”号填列）': 20685.82, '其中：对联营企业和合营企业的投资收益': 0.0, '以摊余成本计量的金融资产终止确认收益（损失以“-”号填列）': 0.0, '净敞口套期收益（损失以“-”号填列）': 0.0, '公允价值变动收益（损失以“-”号填列）': 0.0, '资产处置收益（损失以“-”号填列）': 0.0, '二、营业利润（亏损以“-”号填列）': 1235977.46, '加：营业外收入': 238739.02, '减：营业外支出': 481559.1, '三、利润总额（亏损总额以“-”号填列）': 993157.38, '减：所得税费用': 49644.66, '四、净利润（净亏损以“-”号填列）': 943512.72, '（一）按经营持续性分类': 0.0, '1.持续经营净利润（净亏损以“-”号填列）': 0.0, '2.终止经营净利润（净亏损以“-”号填列）': 0.0, '（二）按所有权归属分类': 0.0, '1.归属于母公司股东的净利润（净亏损以“-”号填列）': 0.0, '2.少数股东损益（净亏损以“-”号填列）': 0.0, '五、其他综合收益的税后净额': 0.0, '（一）不能重分类进损益的其他综合收益': 0.0, '1．重新计量设定受益计划变动额': 0.0, '2．权益法下不能转损益的其他综合收益': 0.0, '3.其他权益工具投资公允价值变动': 0.0, '4.企业自身信用风险公允价值变动': 0.0, '5.其他': 0.0, '（二）将重分类进损益的其他综合收益': 0.0, '1．权益法下可转损益的其他综合收益': 0.0, '2．其他债权投资公允价值变动': 0.0, '3.可供出售金融资产公允价值变动损益': 0.0, '4．金融资产重分类计入其他综合收益的金额': 0.0, '5.持有至到期投资重分类为可供出售的金融资产损益': 0.0, '6．其他债权投资信用减值准备': 0.0, '7．现金流量套期储备（现金流量套期损益的有效部分）': 0.0, '8．外币财务报表折算差额': 0.0, '9.其他': 0.0, '六、综合收益总额': 943512.72, '七、每股收益：': 0.0, '（一）基本每股收益': 0.0, '（二）稀释每股收益': 0.0, '一、经营活动产生的现金流量：': 0.0, '销售商品、提供劳务收到的现金': 68041055.71, '收到的税费返还': 0.0, '收到其他与经营活动有关的现金': -340705.23, '经营活动现金流入小计': 67700350.48, '购买商品、接受劳务支付的现金': 56290400.37, '支付给职工以及为职工支付的现金': 0.0, '支付的各项税费': 0.0, '支付其他与经营活动有关的现金': 0.0, '经营活动现金流出小计': 56290400.37, '经营活动产生的现金流量净额': 6042248.9, '二、投资活动产生的现金流量：': 0.0, '收回投资收到的现金': 750000.0, '取得投资收益收到的现金': 20685.82, '处置固定资产、无形资产和其他长期资产收回的现金净额': -1809733.02, '处置子公司及其他营业单位收到的现金净额': 0.0, '收到其他与投资活动有关的现金': 0.0, '投资活动现金流入小计': -1039047.2, '购建固定资产、无形资产和其他长期资产支付的现金': 0.0, '投资所支付的现金': 0.0, '取得子公司及其他营业单位支付的现金净额': 0.0, '支付其他与投资活动有关的现金': 0.0, '投资活动现金流出小计': 0.0, '投资活动产生的现金流量净额': -1039047.2, '三、筹资活动产生的现金流量：': 0.0, '吸收投资收到的现金': 0.0, '取得借款收到的现金': 0.0, '收到其他与筹资活动有关的现金': 140920.86, '筹资活动现金流入小计': 140920.86, '偿还债务支付的现金': 1450000.0, '分配股利、利润或偿付利息支付的现金': 514038.24, '支付其他与筹资活动有关的现金': 1857750.35, '筹资活动现金流出小计': 3821788.59, '筹资活动产生的现金流量净额': -3680867.73, '四、汇率变动对现金及现金等价物的影响': 0.0, '五、现金及现金等价物净增加额': 6690035.18, '加：期初现金及现金等价物余额': 0.0, '六、期末现金及现金等价物余额': 6690035.18, '现金流量表补充资料如下：': 0.0, '1.将净利润调节为经营活动现金流量：': 0.0, '净利润': 943512.72, '加：资产减值准备': 0.0, '固定资产折旧、油气资产折耗、生产性生物资产折旧、使用权资产折旧、投资性房地产折旧': 2760519.77, '无形资产摊销': 0.0, '长期待摊费用摊销': 0.0, '处置固定资产、无形资产和其他长期资产的损失（收益以“-”号填列）': -1809733.02, '固定资产报废损失（收益以“-”号填列）': 0.0, '公允价值变动损失（收益以“-”号填列）': 0.0, '财务费用（收益以“-”号填列）': 514038.24, '投资损失（收益以“-”号填列）': 20685.82, '递延所得税资产减少（增加以“-”号填列）': 0.0, '递延所得税负债增加（减少以“-”号填列）': 0.0, '存货的减少（增加以“-”号填列）': 1813545.15, '经营性应收项目的减少（增加以“-”号填列）': 2274244.5, '经营性应付项目的增加（减少以“-”号填列）': 991996.48, '其他': 0.0, '2.不涉及现金收支的重大投资和筹资活动：': 0.0, '债务转为资本': 0.0, '一年内到期的可转换公司债券': 0.0, '融资租入固定资产': 0.0, '3.现金及现金等价物净变动情况：': 0.0, '现金的期末余额': 3770116.25, '减：现金的期初余额': 2447782.28, '加：现金等价物的期末余额': 0.0, '减：现金等价物的期初余额': 0.0, '现金及现金等价物净增加额': 1322333.97, 'Head': ['广东周氏神龙电线制造有限公司', [2020, None], '年报']}, {'客户名称': '广东周氏神龙电线制造有限公司', '报表周期': '年报', '报表期次': '2021年', '是否审计': '否', '审计意见': 0.0, '审计意见说明': 0.0, '是否控制名单': '否', '币种': '人民币', '版本': 'V[1]', '报表缺失原因': 0.0, '备注': 0.0, '客户编号': '700008131614', '报表性质': '非合并报表', '录入人': '刘嘉华', '审计日期': 0.0, '审计报告编号': 0.0, None: None, '会计事务所名称': 0.0, '汇率': 0.16, '报表状态': '已生效', '报表模板': '一般企业财务报表(2018版会计准则)', '货币资金': 3117929.54, '交易性金融资产': 0.0, '以公允价值计量且其变动计入当期损益的金融资产': 0.0, '衍生金融资产': 0.0, '应收票据及应收账款': 29334095.7, '应收票据': 4019048.35, '应收账款': 25315047.35, '应收款项融资': 0.0, '预付款项': 0.0, '其他应收款': 8154556.09, '其中：应收利息': 0.0, '应收股利': 0.0, '存货': 2316060.3, '合同资产': 0.0, '持有待售资产': 0.0, '一年内到期的非流动资产': 0.0, '其他流动资产': 4550309.49, '流动资产合计': 47472951.12, '非流动资产：': 0.0, '债权投资': 0.0, '其他债权投资': 0.0, '可供出售金融资产': 0.0, '持有至到期投资': 0.0, '长期应收款': 0.0, '长期股权投资': 0.0, '其他权益工具投资': 0.0, '其他非流动金融资产': 0.0, '投资性房地产': 0.0, '其中：投资性房地产原值': 0.0, '投资性房地产累计折旧': 0.0, '投资性房地产减值准备': 0.0, '固定资产': 2698037.5, '其中：固定资产原值': 8765977.57, '固定资产累计折旧': 6067940.07, '固定资产减值准备': 0.0, '固定资产清理': 0.0, '在建工程': 0.0, '生产性生物资产': 0.0, '其中：生产性生物资产原值': 0.0, '生产性生物资产累计折旧': 0.0, '生产性生物资产减值准备': 0.0, '油气资产': 0.0, '其中：油气资产原值': 0.0, '油气资产累计折耗': 0.0, '油气资产减值准备': 0.0, '使用权资产': 0.0, '其中：使用权资产原值': 0.0, '使用权资产累计折旧': 0.0, '使用权资产减值准备': 0.0, '无形资产': 0.0, '其中：无形资产原值': 0.0, '无形资产累计摊销': 0.0, '无形资产减值准备': 0.0, '开发支出': 0.0, '商誉': 0.0, '长期待摊费用': 0.0, '递延所得税资产': 0.0, '其他非流动资产': 0.0, '非流动资产合计': 2698037.5, '资产总计': 50170988.62, '短期借款': 14800000.0, '交易性金融负债': 0.0, '以公允价值计量且其变动计入当期损益的金融负债': 0.0, '衍生金融负债': 0.0, '应付票据及应付账款': 16643608.76, '其中：应付票据': 0.0, '应付账款': 16643608.76, '预收款项': 0.0, '合同负债': 0.0, '应付职工薪酬': 1070369.0, '应交税费': 771449.06, '其他应付款': 373112.91, '其中：应付利息': 0.0, '应付股利': 0.0, '持有待售负债': 0.0, '一年内到期的非流动负债': 0.0, '其他流动负债': 0.0, '流动负债合计': 33658539.73, '非流动负债：': 0.0, '长期借款': 0.0, '应付债券': 0.0, '其中：优先股': 0.0, '永续债': 0.0, '租赁负债': 0.0, '长期应付款': 0.0, '预计负债': 0.0, '递延收益': 0.0, '递延所得税负债': 0.0, '其他非流动负债': 0.0, '非流动负债合计': 0.0, '负债合计': 33658539.73, '所有者权益（或股东权益）：': 0.0, '实收资本（或股本）': 3000000.0, '其他权益工具': 0.0, '资本公积': 0.0, '减：库存股': 0.0, '其他综合收益': 0.0, '专项储备': 0.0, '盈余公积': 1871491.22, '未分配利润': 11640957.67, '少数股东权益': 0.0, '所有者权益（或股东权益）合计': 16512448.89, '负债和所有者权益（或股东权益）总计': 50170988.62, '一、营业收入': 81605871.8, '减：营业成本': 70022541.21, '税金及附加': 257751.58, '销售费用': 963124.52, '管理费用': 7112561.46, '研发费用': 0.0, '财务费用': 739503.64, '其中:利息费用（本子科目指的是剔除利息资本化部分的；计入损益表的利息费用可通过“财务费用”填列）': 0.0, '利息收入': 0.0, '资产减值损失': 0.0, '信用减值损失': 0.0, '加：其他收益': 0.0, '投资收益（损失以“-”号填列）': 15406.57, '其中：对联营企业和合营企业的投资收益': 0.0, '以摊余成本计量的金融资产终止确认收益（损失以“-”号填列）': 0.0, '净敞口套期收益（损失以“-”号填列）': 0.0, '公允价值变动收益（损失以“-”号填列）': 0.0, '资产处置收益（损失以“-”号填列）': 0.0, '二、营业利润（亏损以“-”号填列）': 2525795.96, '加：营业外收入': 64743.77, '减：营业外支出': 707642.46, '三、利润总额（亏损总额以“-”号填列）': 1882897.27, '减：所得税费用': 136742.64, '四、净利润（净亏损以“-”号填列）': 1746154.63, '（一）按经营持续性分类': 0.0, '1.持续经营净利润（净亏损以“-”号填列）': 0.0, '2.终止经营净利润（净亏损以“-”号填列）': 0.0, '（二）按所有权归属分类': 0.0, '1.归属于母公司股东的净利润（净亏损以“-”号填列）': 0.0, '2.少数股东损益（净亏损以“-”号填列）': 0.0, '五、其他综合收益的税后净额': 0.0, '（一）不能重分类进损益的其他综合收益': 0.0, '1．重新计量设定受益计划变动额': 0.0, '2．权益法下不能转损益的其他综合收益': 0.0, '3.其他权益工具投资公允价值变动': 0.0, '4.企业自身信用风险公允价值变动': 0.0, '5.其他': 0.0, '（二）将重分类进损益的其他综合收益': 0.0, '1．权益法下可转损益的其他综合收益': 0.0, '2．其他债权投资公允价值变动': 0.0, '3.可供出售金融资产公允价值变动损益': 0.0, '4．金融资产重分类计入其他综合收益的金额': 0.0, '5.持有至到期投资重分类为可供出售的金融资产损益': 0.0, '6．其他债权投资信用减值准备': 0.0, '7．现金流量套期储备（现金流量套期损益的有效部分）': 0.0, '8．外币财务报表折算差额': 0.0, '9.其他': 0.0, '六、综合收益总额': 1746154.63, '七、每股收益：': 0.0, '（一）基本每股收益': 0.0, '（二）稀释每股收益': 0.0, '一、经营活动产生的现金流量：': 0.0, '销售商品、提供劳务收到的现金': 83334918.75, '收到的税费返还': 0.0, '收到其他与经营活动有关的现金': -8808107.12, '经营活动现金流入小计': 74526811.63, '购买商品、接受劳务支付的现金': 72461413.61, '支付给职工以及为职工支付的现金': 0.0, '支付的各项税费': -98269.79, '支付其他与经营活动有关的现金': 8783328.44, '经营活动现金流出小计': 81146472.26, '经营活动产生的现金流量净额': -6619660.63, '二、投资活动产生的现金流量：': 0.0, '收回投资收到的现金': 570000.0, '取得投资收益收到的现金': 15406.57, '处置固定资产、无形资产和其他长期资产收回的现金净额': 0.0, '处置子公司及其他营业单位收到的现金净额': 0.0, '收到其他与投资活动有关的现金': 0.0, '投资活动现金流入小计': 585406.57, '购建固定资产、无形资产和其他长期资产支付的现金': 1674388.67, '投资所支付的现金': 15406.57, '取得子公司及其他营业单位支付的现金净额': 0.0, '支付其他与投资活动有关的现金': 0.0, '投资活动现金流出小计': 1689795.24, '投资活动产生的现金流量净额': -1104388.67, '三、筹资活动产生的现金流量：': 0.0, '吸收投资收到的现金': 0.0, '取得借款收到的现金': 8800000.0, '收到其他与筹资活动有关的现金': 0.0, '筹资活动现金流入小计': 8800000.0, '偿还债务支付的现金': 0.0, '分配股利、利润或偿付利息支付的现金': 739503.64, '支付其他与筹资活动有关的现金': 1004040.34, '筹资活动现金流出小计': 1743543.98, '筹资活动产生的现金流量净额': 7056456.02, '四、汇率变动对现金及现金等价物的影响': 0.0, '五、现金及现金等价物净增加额': -667593.28, '加：期初现金及现金等价物余额': 0.0, '六、期末现金及现金等价物余额': -667593.28, '现金流量表补充资料如下：': 0.0, '1.将净利润调节为经营活动现金流量：': 0.0, '净利润': 1746154.63, '加：资产减值准备': 0.0, '固定资产折旧、油气资产折耗、生产性生物资产折旧、使用权资产折旧、投资性房地产折旧': 502765.84, '无形资产摊销': 0.0, '长期待摊费用摊销': 1877278.4, '处置固定资产、无形资产和其他长期资产的损失（收益以“-”号填列）': -1674388.67, '固定资产报废损失（收益以“-”号填列）': 0.0, '公允价值变动损失（收益以“-”号填列）': 0.0, '财务费用（收益以“-”号填列）': 739503.64, '投资损失（收益以“-”号填列）': 15406.57, '递延所得税资产减少（增加以“-”号填列）': 0.0, '递延所得税负债增加（减少以“-”号填列）': 0.0, '存货的减少（增加以“-”号填列）': -1765434.52, '经营性应收项目的减少（增加以“-”号填列）': -3181186.04, '经营性应付项目的增加（减少以“-”号填列）': -95748.12, '其他': 0.0, '2.不涉及现金收支的重大投资和筹资活动：': 0.0, '债务转为资本': 0.0, '一年内到期的可转换公司债券': 0.0, '融资租入固定资产': 0.0, '3.现金及现金等价物净变动情况：': 0.0, '现金的期末余额': 3117929.54, '减：现金的期初余额': 3770116.25, '加：现金等价物的期末余额': 0.0, '减：现金等价物的期初余额': 0.0, '现金及现金等价物净增加额': -652186.71, 'Head': ['广东周氏神龙电线制造有限公司', [2021, None], '年报']}, {'客户名称': '广东周氏神龙电线制造有限公司', '报表周期': '年报', '报表期次': '2022年', '是否审计': '否', '审计意见': 0.0, '审计意见说明': 0.0, '是否控制名单': '', '币种': 0.0, '版本': 'V[1]', '报表缺失原因': 0.0, '备注': 0.0, '客户编号': '700008131614', '报表性质': '非合并报表', '录入人': '刘嘉华', '审计日期': 0.0, '审计报告编号': 0.0, None: None, '会计事务所名称': 0.0, '汇率': 0.14, '报表状态': '已生效', '报表模板': '一般企业财务报表(2018版会计准则)', '货币资金': 3378540.84, '交易性金融资产': 0.0, '以公允价值计量且其变动计入当期损益的金融资产': 0.0, '衍生金融资产': 0.0, '应收票据及应收账款': 30901102.02, '应收票据': 4687810.07, '应收账款': 26213291.95, '应收款项融资': 0.0, '预付款项': 0.0, '其他应收款': 8408670.14, '其中：应收利息': 0.0, '应收股利': 0.0, '存货': 2665826.85, '合同资产': 0.0, '持有待售资产': 0.0, '一年内到期的非流动资产': 0.0, '其他流动资产': 5415007.45, '流动资产合计': 50769147.3, '非流动资产：': 0.0, '债权投资': 0.0, '其他债权投资': 0.0, '可供出售金融资产': 0.0, '持有至到期投资': 0.0, '长期应收款': 0.0, '长期股权投资': 0.0, '其他权益工具投资': 0.0, '其他非流动金融资产': 0.0, '投资性房地产': 0.0, '其中：投资性房地产原值': 0.0, '投资性房地产累计折旧': 0.0, '投资性房地产减值准备': 0.0, '固定资产': 2396663.92, '其中：固定资产原值': 9161129.05, '固定资产累计折旧': 6764465.13, '固定资产减值准备': 0.0, '固定资产清理': 0.0, '在建工程': 0.0, '生产性生物资产': 0.0, '其中：生产性生物资产原值': 0.0, '生产性生物资产累计折旧': 0.0, '生产性生物资产减值准备': 0.0, '油气资产': 0.0, '其中：油气资产原值': 0.0, '油气资产累计折耗': 0.0, '油气资产减值准备': 0.0, '使用权资产': 0.0, '其中：使用权资产原值': 0.0, '使用权资产累计折旧': 0.0, '使用权资产减值准备': 0.0, '无形资产': 0.0, '其中：无形资产原值': 0.0, '无形资产累计摊销': 0.0, '无形资产减值准备': 0.0, '开发支出': 0.0, '商誉': 0.0, '长期待摊费用': 0.0, '递延所得税资产': 0.0, '其他非流动资产': 0.0, '非流动资产合计': 2396663.92, '资产总计': 53165811.22, '短期借款': 12000000.0, '交易性金融负债': 0.0, '以公允价值计量且其变动计入当期损益的金融负债': 0.0, '衍生金融负债': 0.0, '应付票据及应付账款': 15496854.89, '其中：应付票据': 0.0, '应付账款': 15496854.89, '预收款项': 0.0, '合同负债': 0.0, '应付职工薪酬': 979775.0, '应交税费': 1346721.48, '其他应付款': 1739307.0, '其中：应付利息': 0.0, '应付股利': 0.0, '持有待售负债': 0.0, '一年内到期的非流动负债': 0.0, '其他流动负债': 0.0, '流动负债合计': 31562658.37, '非流动负债：': 0.0, '长期借款': 2925000.0, '应付债券': 0.0, '其中：优先股': 0.0, '永续债': 0.0, '租赁负债': 0.0, '长期应付款': 0.0, '预计负债': 0.0, '递延收益': 0.0, '递延所得税负债': 0.0, '其他非流动负债': 0.0, '非流动负债合计': 2925000.0, '负债合计': 34487658.37, '所有者权益（或股东权益）：': 0.0, '实收资本（或股本）': 3000000.0, '其他权益工具': 0.0, '资本公积': 0.0, '减：库存股': 0.0, '其他综合收益': 0.0, '专项储备': 0.0, '盈余公积': 2196346.82, '未分配利润': 13481806.03, '少数股东权益': 0.0, '所有者权益（或股东权益）合计': 18678152.85, '负债和所有者权益（或股东权益）总计': 53165811.22, '一、营业收入': 83502389.47, '减：营业成本': 72538699.47, '税金及附加': 163297.69, '销售费用': 735898.05, '管理费用': 6813084.36, '研发费用': 0.0, '财务费用': 857393.83, '其中:利息费用（本子科目指的是剔除利息资本化部分的；计入损益表的利息费用可通过“财务费用”填列）': 0.0, '利息收入': 0.0, '资产减值损失': 0.0, '信用减值损失': 0.0, '加：其他收益': 0.0, '投资收益（损失以“-”号填列）': 2268.77, '其中：对联营企业和合营企业的投资收益': 0.0, '以摊余成本计量的金融资产终止确认收益（损失以“-”号填列）': 0.0, '净敞口套期收益（损失以“-”号填列）': 0.0, '公允价值变动收益（损失以“-”号填列）': 0.0, '资产处置收益（损失以“-”号填列）': 0.0, '二、营业利润（亏损以“-”号填列）': 2396284.84, '加：营业外收入': 55343.98, '减：营业外支出': 110788.03, '三、利润总额（亏损总额以“-”号填列）': 2340840.79, '减：所得税费用': 364708.57, '四、净利润（净亏损以“-”号填列）': 1976132.22, '（一）按经营持续性分类': 0.0, '1.持续经营净利润（净亏损以“-”号填列）': 0.0, '2.终止经营净利润（净亏损以“-”号填列）': 0.0, '（二）按所有权归属分类': 0.0, '1.归属于母公司股东的净利润（净亏损以“-”号填列）': 0.0, '2.少数股东损益（净亏损以“-”号填列）': 0.0, '五、其他综合收益的税后净额': 0.0, '（一）不能重分类进损益的其他综合收益': 0.0, '1．重新计量设定受益计划变动额': 0.0, '2．权益法下不能转损益的其他综合收益': 0.0, '3.其他权益工具投资公允价值变动': 0.0, '4.企业自身信用风险公允价值变动': 0.0, '5.其他': 0.0, '（二）将重分类进损益的其他综合收益': 0.0, '1．权益法下可转损益的其他综合收益': 0.0, '2．其他债权投资公允价值变动': 0.0, '3.可供出售金融资产公允价值变动损益': 0.0, '4．金融资产重分类计入其他综合收益的金额': 0.0, '5.持有至到期投资重分类为可供出售的金融资产损益': 0.0, '6．其他债权投资信用减值准备': 0.0, '7．现金流量套期储备（现金流量套期损益的有效部分）': 0.0, '8．外币财务报表折算差额': 0.0, '9.其他': 0.0, '六、综合收益总额': 1976132.22, '七、每股收益：': 0.0, '（一）基本每股收益': 0.0, '（二）稀释每股收益': 0.0, '一、经营活动产生的现金流量：': 0.0, '销售商品、提供劳务收到的现金': 81935383.15, '收到的税费返还': 0.0, '收到其他与经营活动有关的现金': 1667424.02, '经营活动现金流入小计': 83602807.17, '购买商品、接受劳务支付的现金': 74793986.79, '支付给职工以及为职工支付的现金': 0.0, '支付的各项税费': -47266.16, '支付其他与经营活动有关的现金': 7659770.44, '经营活动现金流出小计': 82406491.07, '经营活动产生的现金流量净额': 384307.2, '二、投资活动产生的现金流量：': 0.0, '收回投资收到的现金': 0.0, '取得投资收益收到的现金': 2268.77, '处置固定资产、无形资产和其他长期资产收回的现金净额': 0.0, '处置子公司及其他营业单位收到的现金净额': 0.0, '收到其他与投资活动有关的现金': 0.0, '投资活动现金流入小计': 2268.77, '购建固定资产、无形资产和其他长期资产支付的现金': 395151.48, '投资所支付的现金': 0.0, '取得子公司及其他营业单位支付的现金净额': 0.0, '支付其他与投资活动有关的现金': 0.0, '投资活动现金流出小计': 395151.48, '投资活动产生的现金流量净额': -392882.71, '三、筹资活动产生的现金流量：': 0.0, '吸收投资收到的现金': 0.0, '取得借款收到的现金': 125000.0, '收到其他与筹资活动有关的现金': 324855.6, '筹资活动现金流入小计': 449855.6, '偿还债务支付的现金': 0.0, '分配股利、利润或偿付利息支付的现金': 857393.83, '支付其他与筹资活动有关的现金': 135283.86, '筹资活动现金流出小计': 992677.69, '筹资活动产生的现金流量净额': -542822.09, '四、汇率变动对现金及现金等价物的影响': 0.0, '五、现金及现金等价物净增加额': 260611.3, '加：期初现金及现金等价物余额': 0.0, '六、期末现金及现金等价物余额': 260611.3, '现金流量表补充资料如下：': 0.0, '1.将净利润调节为经营活动现金流量：': 0.0, '净利润': 1976132.22, '加：资产减值准备': 0.0, '固定资产折旧、油气资产折耗、生产性生物资产折旧、使用权资产折旧、投资性房地产折旧': 696525.06, '无形资产摊销': 0.0, '长期待摊费用摊销': 0.0, '处置固定资产、无形资产和其他长期资产的损失（收益以“-”号填列）': -349766.55, '固定资产报废损失（收益以“-”号填列）': 0.0, '公允价值变动损失（收益以“-”号填列）': 0.0, '财务费用（收益以“-”号填列）': 0.0, '投资损失（收益以“-”号填列）': 2268.77, '递延所得税资产减少（增加以“-”号填列）': 0.0, '递延所得税负债增加（减少以“-”号填列）': 0.0, '存货的减少（增加以“-”号填列）': -349766.55, '经营性应收项目的减少（增加以“-”号填列）': 2729189.12, '经营性应付项目的增加（减少以“-”号填列）': 704118.64, '其他': 0.0, '2.不涉及现金收支的重大投资和筹资活动：': 0.0, '债务转为资本': 0.0, '一年内到期的可转换公司债券': 0.0, '融资租入固定资产': 0.0, '3.现金及现金等价物净变动情况：': 0.0, '现金的期末余额': 3378540.84, '减：现金的期初余额': 3117929.54, '加：现金等价物的期末余额': 0.0, '减：现金等价物的期初余额': 0.0, '现金及现金等价物净增加额': 260611.3, 'Head': ['广东周氏神龙电线制造有限公司', [2022, None], '年报']}, {'客户名称': '广东周氏神龙电线制造有限公司', '报表周期': '月报', '报表期次': '2023年5月', '是否审计': '否', '审计意见': 0.0, '审计意见说明': 0.0, '是否控制名单': '否', '币种': '人民币', '版本': 'V[1]', '报表缺失原因': 0.0, '备注': 0.0, '客户编号': '700008131614', '报表性质': '非合并报表', '录入人': '刘嘉华', '审计日期': 0.0, '审计报告编号': 0.0, None: None, '会计事务所名称': 0.0, '汇率': 0.14, '报表状态': '已生效', '报表模板': '一般企业财务报表(2018版会计准则)', '货币资金': 3108743.77, '交易性金融资产': 0.0, '以公允价值计量且其变动计入当期损益的金融资产': 0.0, '衍生金融资产': 0.0, '应收票据及应收账款': 34173108.26, '应收票据': 4603471.32, '应收账款': 29569636.94, '应收款项融资': 0.0, '预付款项': 0.0, '其他应收款': 8453231.59, '其中：应收利息': 0.0, '应收股利': 0.0, '存货': 1503064.71, '合同资产': 0.0, '持有待售资产': 0.0, '一年内到期的非流动资产': 0.0, '其他流动资产': 5687215.8, '流动资产合计': 52925364.13, '非流动资产：': 0.0, '债权投资': 0.0, '其他债权投资': 0.0, '可供出售金融资产': 0.0, '持有至到期投资': 0.0, '长期应收款': 0.0, '长期股权投资': 0.0, '其他权益工具投资': 0.0, '其他非流动金融资产': 0.0, '投资性房地产': 0.0, '其中：投资性房地产原值': 0.0, '投资性房地产累计折旧': 0.0, '投资性房地产减值准备': 0.0, '固定资产': 2178506.14, '其中：固定资产原值': 9183290.74, '固定资产累计折旧': 7004784.6, '固定资产减值准备': 0.0, '固定资产清理': 0.0, '在建工程': 0.0, '生产性生物资产': 0.0, '其中：生产性生物资产原值': 0.0, '生产性生物资产累计折旧': 0.0, '生产性生物资产减值准备': 0.0, '油气资产': 0.0, '其中：油气资产原值': 0.0, '油气资产累计折耗': 0.0, '油气资产减值准备': 0.0, '使用权资产': 0.0, '其中：使用权资产原值': 0.0, '使用权资产累计折旧': 0.0, '使用权资产减值准备': 0.0, '无形资产': 0.0, '其中：无形资产原值': 0.0, '无形资产累计摊销': 0.0, '无形资产减值准备': 0.0, '开发支出': 0.0, '商誉': 0.0, '长期待摊费用': 0.0, '递延所得税资产': 0.0, '其他非流动资产': 0.0, '非流动资产合计': 2178506.14, '资产总计': 55103870.27, '短期借款': 0.0, '交易性金融负债': 14000000.0, '以公允价值计量且其变动计入当期损益的金融负债': 0.0, '衍生金融负债': 0.0, '应付票据及应付账款': 16209892.04, '其中：应付票据': 0.0, '应付账款': 16209892.04, '预收款项': 0.0, '合同负债': 0.0, '应付职工薪酬': 353972.0, '应交税费': 216055.96, '其他应付款': 1944129.63, '其中：应付利息': 0.0, '应付股利': 0.0, '持有待售负债': 0.0, '一年内到期的非流动负债': 0.0, '其他流动负债': 0.0, '流动负债合计': 32724049.63, '非流动负债：': 0.0, '长期借款': 2850000.0, '应付债券': 0.0, '其中：优先股': 0.0, '永续债': 0.0, '租赁负债': 0.0, '长期应付款': 0.0, '预计负债': 0.0, '递延收益': 0.0, '递延所得税负债': 0.0, '其他非流动负债': 0.0, '非流动负债合计': 2850000.0, '负债合计': 35574049.63, '所有者权益（或股东权益）：': 0.0, '实收资本（或股本）': 3000000.0, '其他权益工具': 0.0, '资本公积': 0.0, '减：库存股': 0.0, '其他综合收益': 0.0, '专项储备': 0.0, '盈余公积': 2196346.82, '未分配利润': 14333473.82, '少数股东权益': 0.0, '所有者权益（或股东权益）合计': 19529820.64, '负债和所有者权益（或股东权益）总计': 55103870.27, '一、营业收入': 32744544.7, '减：营业成本': 28845433.43, '税金及附加': 52116.22, '销售费用': 228071.99, '管理费用': 2434081.71, '研发费用': 0.0, '财务费用': 288789.48, '其中:利息费用（本子科目指的是剔除利息资本化部分的；计入损益表的利息费用可通过“财务费用”填列）': 0.0, '利息收入': 0.0, '资产减值损失': 0.0, '信用减值损失': 0.0, '加：其他收益': 0.0, '投资收益（损失以“-”号填列）': 0.0, '其中：对联营企业和合营企业的投资收益': 0.0, '以摊余成本计量的金融资产终止确认收益（损失以“-”号填列）': 0.0, '净敞口套期收益（损失以“-”号填列）': 0.0, '公允价值变动收益（损失以“-”号填列）': 0.0, '资产处置收益（损失以“-”号填列）': 0.0, '二、营业利润（亏损以“-”号填列）': 896051.87, '加：营业外收入': 35249.48, '减：营业外支出': 540.0, '三、利润总额（亏损总额以“-”号填列）': 930761.35, '减：所得税费用': 79093.56, '四、净利润（净亏损以“-”号填列）': 851667.79, '（一）按经营持续性分类': 0.0, '1.持续经营净利润（净亏损以“-”号填列）': 0.0, '2.终止经营净利润（净亏损以“-”号填列）': 0.0, '（二）按所有权归属分类': 0.0, '1.归属于母公司股东的净利润（净亏损以“-”号填列）': 0.0, '2.少数股东损益（净亏损以“-”号填列）': 0.0, '五、其他综合收益的税后净额': 0.0, '（一）不能重分类进损益的其他综合收益': 0.0, '1．重新计量设定受益计划变动额': 0.0, '2．权益法下不能转损益的其他综合收益': 0.0, '3.其他权益工具投资公允价值变动': 0.0, '4.企业自身信用风险公允价值变动': 0.0, '5.其他': 0.0, '（二）将重分类进损益的其他综合收益': 0.0, '1．权益法下可转损益的其他综合收益': 0.0, '2．其他债权投资公允价值变动': 0.0, '3.可供出售金融资产公允价值变动损益': 0.0, '4．金融资产重分类计入其他综合收益的金额': 0.0, '5.持有至到期投资重分类为可供出售的金融资产损益': 0.0, '6．其他债权投资信用减值准备': 0.0, '7．现金流量套期储备（现金流量套期损益的有效部分）': 0.0, '8．外币财务报表折算差额': 0.0, '9.其他': 0.0, '六、综合收益总额': 851667.79, '七、每股收益：': 0.0, '（一）基本每股收益': 0.0, '（二）稀释每股收益': 0.0, '一、经营活动产生的现金流量：': 0.0, '销售商品、提供劳务收到的现金': 5108946.14, '收到的税费返还': 0.0, '收到其他与经营活动有关的现金': 1305996.42, '经营活动现金流入小计': 6414942.56, '购买商品、接受劳务支付的现金': 5757606.88, '支付给职工以及为职工支付的现金': 0.0, '支付的各项税费': 540895.77, '支付其他与经营活动有关的现金': 1251873.9, '经营活动现金流出小计': 7550376.55, '经营活动产生的现金流量净额': -1135433.99, '二、投资活动产生的现金流量：': 0.0, '收回投资收到的现金': 0.0, '取得投资收益收到的现金': 0.0, '处置固定资产、无形资产和其他长期资产收回的现金净额': 0.0, '处置子公司及其他营业单位收到的现金净额': 0.0, '收到其他与投资活动有关的现金': 0.0, '投资活动现金流入小计': 0.0, '购建固定资产、无形资产和其他长期资产支付的现金': 22161.69, '投资所支付的现金': 0.0, '取得子公司及其他营业单位支付的现金净额': 0.0, '支付其他与投资活动有关的现金': 0.0, '投资活动现金流出小计': 22161.69, '投资活动产生的现金流量净额': -22161.69, '三、筹资活动产生的现金流量：': 0.0, '吸收投资收到的现金': -2000000.0, '取得借款收到的现金': 0.0, '收到其他与筹资活动有关的现金': -1343704.19, '筹资活动现金流入小计': -3343704.19, '偿还债务支付的现金': 925000.0, '分配股利、利润或偿付利息支付的现金': 8625.71, '支付其他与筹资活动有关的现金': 4433695.68, '筹资活动现金流出小计': 5367321.39, '筹资活动产生的现金流量净额': -8711025.58, '四、汇率变动对现金及现金等价物的影响': 0.0, '五、现金及现金等价物净增加额': -9868621.26, '加：期初现金及现金等价物余额': 0.0, '六、期末现金及现金等价物余额': -9868621.26, '现金流量表补充资料如下：': 0.0, '1.将净利润调节为经营活动现金流量：': 0.0, '净利润': 87199.19, '加：资产减值准备': 0.0, '固定资产折旧、油气资产折耗、生产性生物资产折旧、使用权资产折旧、投资性房地产折旧': 240319.47, '无形资产摊销': 0.0, '长期待摊费用摊销': 0.0, '处置固定资产、无形资产和其他长期资产的损失（收益以“-”号填列）': -22161.69, '固定资产报废损失（收益以“-”号填列）': 0.0, '公允价值变动损失（收益以“-”号填列）': 0.0, '财务费用（收益以“-”号填列）': 8625.71, '投资损失（收益以“-”号填列）': 0.0, '递延所得税资产减少（增加以“-”号填列）': 0.0, '递延所得税负债增加（减少以“-”号填列）': 0.0, '存货的减少（增加以“-”号填列）': 1162762.14, '经营性应收项目的减少（增加以“-”号填列）': 2098439.76, '经营性应付项目的增加（减少以“-”号填列）': 954435.54, '其他': 0.0, '2.不涉及现金收支的重大投资和筹资活动：': 0.0, '债务转为资本': 0.0, '一年内到期的可转换公司债券': 0.0, '融资租入固定资产': 0.0, '3.现金及现金等价物净变动情况：': 0.0, '现金的期末余额': 3108743.77, '减：现金的期初余额': 3378540.84, '加：现金等价物的期末余额': 0.0, '减：现金等价物的期初余额': 0.0, '现金及现金等价物净增加额': -269797.07, 'Head': ['广东周氏神龙电线制造有限公司', [2023, None], '月报']}]
assetSubjects = ['货币资金','交易性金融资产','以公允价值计量且其变动计入当期损益的金融资产','衍生金融资产','应收票据','应收账款','应收款项融资','预付款项','其他应收款','存货','合同资产','持有待售资产','一年内到期的非流动资产','其他流动资产','债权投资','其他债权投资','可供出售金融资产','持有至到期投资','长期应收款','长期股权投资','其他权益工具投资','其他非流动金融资产','投资性房地产','固定资产','在建工程','生产性生物资产','油气资产','使用权资产','无形资产','开发支出','商誉','长期待摊费用','递延所得税资产','其他非流动资产']
debetSubujects = ['短期借款','交易性金融负债','应付票据','应付账款','预收款项','合同负债','应付职工薪酬','应交税费','其他应付款','持有待售负债','一年内到期的非流动负债','其他流动负债','长期借款','应付债券','租赁负债','长期应付款','预计负债','递延收益','递延所得税负债','其他非流动负债']
netAssetSubjects = ['实收资本（或股本）','其他权益工具','资本公积','减：库存股','其他综合收益','专项储备','盈余公积','未分配利润','少数股东权益']

class Fi:
    # 定义Fi类，传入财报列表构建
    def __init__(self, filist):
        self.detail = filist

    # SortAll方法，无实参，从小到大对列表进行排序，排序依据为字典中Head对应的年份（Head-dates-year）
    def SortAll(self):
        sorted(self.detail, key = lambda x:x['Head'][1][0])
   
    # ShowDeatil方法，无实参，返回目前状态的财报列表
    def ShowDetail(self):
        return self.detail

    # Shijian方法，无实参，获取几期财报的时间区段
    def Shijian(self):
        date = self.detail[0]['报表期次'] + "度至" + self.detail[-1]['报表期次'] + "," 
        return date

    # Shidian方法，无实参，获取几期财报的期末时点区段
    def Shidian(self):
        date = self.detail[0]['报表期次'] + "末至" + self.detail[-1]['报表期次'] + "末," 
        return date      
    
    def Customer(self):
        customerName = self.detail[0]['客户名称']
        return customerName

    # Assets方法，获取仅含资产类科目的财报字典列表，字典的键中除资产类科目之外仅有head
    def Assets(self):
        assets = []
        for e in self.detail:
            asset = {}
            asset['Head'] = e['Head']
            for key, value in e.items():
                if key in assetSubjects:
                    asset[key] = value
            assets.append(asset)
        return assets
    
    #CertainAssetRemoveZero方法，传入具体期次的序号，输出对应期次的资产科目字典（仅含资产科目）
    def CertainAssetRemoveZero(self,i):
        assets = []
        for e in self.detail:
            asset = {}
            for key, value in e.items():
                if key in assetSubjects:
                    asset[key] = value
            assets.append(asset)
        asset = assets[i]
        keys_to_delete = [key for key, value in asset.items() if value == 0]
        for key in keys_to_delete:
            del asset[key]
        return asset

    # SearchSubject方法，传入待搜索的科目名称（str）参数，输出该科目数值列表（float[])
    def SearchSubject(self, sub):
        values = []
        for e in self.detail:
            values.append(e[sub])
        return values
    
    # SearchSubjecti方法，传入待搜索的科目名称（str）参数，输出该科目数值列表（int[])
    def SearchSubjecti(self, sub):
        values = []
        for e in self.detail:
            values.append(int(e[sub]))
        return values

    # SearchSubjectw方法，传入待搜索的科目名称（str）参数、以及一个类round位数的参数（.0\.2），输出该科目数值万元列表（str[])  
    def SearchSubjectw(self,sub,dotty):
        outValuesw = []
        values = self.SearchSubject(sub)
        for v in values:
            if isinstance(v,(float)):
                if dotty == '.2':outValuesw.append(str(round(v/10000,2))+'万元')
                if dotty == '.0':outValuesw.append(str(int(v/10000))+'万元')
        if outValuesw is not None:
            return outValuesw
        else:return 'Warning:传入列表非数字形，无法完成'
    
    def DoAssetAnal(self):
        self.CertainAssetRemoveZero(-1)

    
def DrawBalancePie(asset):
    # 设置字体为黑体，以支持中文显示
    my_font = font_manager.FontProperties(fname='/Library/Fonts/Songti.ttc')
    plt.rcParams['font.sans-serif'] = ['Monaco']
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    # 创建画布和轴
    fig, ax = plt.subplots()
    # 绘制饼图
    ax.pie(list(asset.values()), labels=list(asset.keys()), autopct='%1.1f%%', startangle=140)
    # 设置标题
    ax.set_title('资产分布')
    # 确保饼图是圆形的
    plt.axis('equal')
    # 显示图表
    plt.show()
                
fi = Fi(filist)
fi.SortAll()
print(fi.ShowDetail())
print(fi.SearchSubject('货币资金'))
print(fi.SearchSubject('一、营业收入'))
print(fi.SearchSubjectw('一、营业收入','.0'))
print(fi.SearchSubjectw('一、营业收入','.2'))
asset = fi.CertainAssetRemoveZero(-1)
print(asset)
assetSum = 0
for key, value in asset.items():
    assetSum += value
print(assetSum)
# DrawBalancePie(asset)
print(fi.Shijian())


[{'客户名称': '广东周氏神龙电线制造有限公司', '报表周期': '年报', '报表期次': '2020年', '是否审计': '否', '审计意见': 0.0, '审计意见说明': 0.0, '是否控制名单': '否', '币种': '人民币', '版本': 'V[1]', '报表缺失原因': 0.0, '备注': 0.0, '客户编号': '700008131614', '报表性质': '非合并报表', '录入人': '刘嘉华', '审计日期': 0.0, '审计报告编号': 0.0, None: None, '会计事务所名称': 0.0, '汇率': 0.15, '报表状态': '已生效', '报表模板': '一般企业财务报表(2018版会计准则)', '货币资金': 3770116.25, '交易性金融资产': 570000.0, '以公允价值计量且其变动计入当期损益的金融资产': 0.0, '衍生金融资产': 0.0, '应收票据及应收账款': 31063142.65, '应收票据': 2980257.35, '应收账款': 28082885.3, '应收款项融资': 0.0, '预付款项': 0.0, '其他应收款': 9400.96, '其中：应收利息': 0.0, '应收股利': 0.0, '存货': 550625.78, '合同资产': 0.0, '持有待售资产': 0.0, '一年内到期的非流动资产': 0.0, '其他流动资产': 3234922.14, '流动资产合计': 39198207.78, '非流动资产：': 0.0, '债权投资': 0.0, '其他债权投资': 0.0, '可供出售金融资产': 0.0, '持有至到期投资': 0.0, '长期应收款': 0.0, '长期股权投资': 0.0, '其他权益工具投资': 0.0, '其他非流动金融资产': 0.0, '投资性房地产': 0.0, '其中：投资性房地产原值': 0.0, '投资性房地产累计折旧': 0.0, '投资性房地产减值准备': 0.0, '固定资产': 1526414.67, '其中：固定资产原值': 7091588.9, '固定资产累计折旧': 5565174.23, '固定资产减值准备': 0.0, '固定资产清理': 0.0, '在建工程': 0.0,

可视化图形的模块